<a href="https://colab.research.google.com/github/gt-cse-6040/skills_oh_week_09/blob/main/math_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Translating math to code

In [1]:
import numpy as np

## Pearson Correlation Coefficient

Given two vectors $x$ and $y$ of the same length we define the Pearson correlation coefficient $r$ as follows:

$$r = \frac{\sum_{i=1}^n(x_i - \bar{x})(y_i-\bar{y})}{\sqrt{\sum_{i=1}^n(x_i - \bar{x})^2\sum_{i=1}^n(y_i - \bar{y})^2}}$$

We want to implement this formula in Python...



In [2]:
def pearson_correlation(x, y):
  n = len(x)
  x_bar = sum(x)/n
  y_bar = sum(y)/n
  res_x = [xi - x_bar for xi in x]
  res_y = [yi - y_bar for yi in y]
  numerator = sum(xi*yi for xi, yi in zip(res_x, res_y))
  denominator_squared = sum(xi**2 for xi in res_x) * sum(yi**2 for yi in res_y)
  denominator = denominator_squared**(1/2)
  return numerator/denominator

This is a little hard to parse, but it *should* work. Let's do an ad-hoc test and compare to `numpy`'s built-in function for the same calculation. Here we will take 100 thousand randomly generated numbers and see how much the calculations diverge.

In [14]:
some_x = np.random.random(100000) * 10
some_y = np.random.random(100000) * 10

In [15]:
numpy_answer = np.corrcoef(some_x, some_y)[0,1]
our_answer = pearson_correlation(some_x, some_y)
abs(numpy_answer-our_answer)

1.7997756063259374e-17

Not very much difference $≈2×10^{-17}$ But how fast is it?

In [25]:
python_time = %timeit -o pearson_correlation(some_x, some_y)
builtin_time = %timeit -o np.corrcoef(some_x, some_y)[0,1]
python_time.average / builtin_time.average

265 ms ± 10.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
877 µs ± 31.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


302.41921354115993

The builtin is *hundreds* of times faster, and our solution is kind of hard to read. Let's think of how we can improve it!

* Is there any way to "vectorize" the formula?
  * Improves efficiency by using optimized Numpy operations
  * Improves readability

Yes! We can vectorize this formula. If we define $\overset{*}{x}$ such that $\overset{*}{x}_i = x_i - \bar{x}$ then our formula becomes much simpler to write in terms of **dot products**.
$$r = \frac{\overset{*}{x}^T\overset{*}{y}}{\sqrt{\overset{*}{x}^T\overset{*}{x}\overset{*}{y}^T\overset{*}{y}}}$$



In [26]:
def vector_pearson(x, y):
  x_star = x - np.mean(x)
  y_star = y - np.mean(y)
  numerator = x_star.dot(y_star)
  denominator = np.sqrt(x_star.dot(x_star) * y_star.dot(y_star))
  return numerator/denominator

In [28]:
numpy_answer = np.corrcoef(some_x, some_y)[0,1]
our_answer = vector_pearson(some_x, some_y)
abs(numpy_answer-our_answer)

1.1492543028346347e-17

In [27]:
python_time = %timeit -o vector_pearson(some_x, some_y)
builtin_time = %timeit -o np.corrcoef(some_x, some_y)[0,1]
python_time.average / builtin_time.average

466 µs ± 74.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
869 µs ± 12.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


0.5355100630694728

Turns out this implementation is faster! (don't get too excited `corrcoef` is calculating multiple coefficients, but we're only looking at one)

Let's review
- We can translate $\sum_{\forall i} v_i * w_i = v^Tw$ for any vectors $v$ and $w$ of the same length.
- We can perform element-wise and broadcasting arithmetec using mathematical operators (+-*/)
